### Load necessary libraries

In [1]:
import pandas as pd

### Load the training and test datasets

In [2]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Extract test passenger IDs for submission

In [3]:
test_ids = test["PassengerId"]

### Define a function to clean the data

In [4]:
def clean(data):
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
    data.Embarked.fillna("U", inplace=True)
    return data

### Apply the cleaning function to both training and test datasets

In [5]:
data = clean(data)
test = clean(test)
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


### **Label encoding for categorical variables**

In [6]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]

for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    print(le.classes_)

data.head(5)


['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


# **Model training**

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### Define features (X) and the target variable (y)

In [8]:
y = data["Survived"]
X = data.drop("Survived", axis=1)

### Split the data into training and testing sets

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Train a logistic regression model

In [10]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

# **Model evaluation**

In [11]:
predictions = clf.predict(X_test)

### Calculate accuracy

In [12]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, predictions)

0.8100558659217877

### Create a submission file

In [15]:
submission_preds = clf.predict(test)

df = pd.DataFrame({"PassengerId": test_ids.values, "Survived": submission_preds})

df.to_csv("submission.csv", index=False)

pd.read_csv("submission.csv").sample(5)

,PassengerId,Survived
320,1212,0
273,1165,1
244,1136,0
365,1257,0
110,1002,0


## **Save the trained model using pickle**

In [14]:
import pickle as pkl

pkl.dump(clf, open("titanic_model.pkl", "wb"))